In [ ]:
import openai
import pandas as pd
import time
import requests
import json
import jsonlines
import re

sk = "YOUR API KEY"
openai.api_key = sk

def chat(input_data, model="gpt-3.5-turbo", temperature=0.8):
    
    nmessages = [{"role": "user", "content": input_data, "temperature": temperature}]

    while (1):
        try:
            response = openai.ChatCompletion.create(          
                model=model,
                messages=nmessages
            )
            resmessage = response['choices'][0]['message']['content']
            break
        except:
            time.sleep(10)
            
    return  resmessage

## Without Retrieved Data

In [ ]:
processed_tuples = []
with open('./results/tv/GPT4_tv_wo_evidence.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

template = '''What's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {answer_format}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''

# template = "You are provided with a table that contains some missing denoted by 'N/A'. Your task is to fill in these missing values by replacing each 'N/A' with the correct value. The output should be a complete table, with the missing values filled in, maintaining the original format, i.e. using '|' to seperate each cell.\n"
tuples = {}

with open('/Users/yichendezaizai/Data_Imputation/data/show_movie/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))


# 读取表格数据tv_ret_1.csv
tableData = []
ground_truth = []
df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/show_movie/tv_ret_1.csv')


count, acc = 0, 0

example = ['Avatar: The Last Airbender', '2005', 'TV-Y7', '9.2' , '1']
for index, row in df.iterrows():

    if index not in test_qids or index in processed_tuples:
        continue

    input_data = template + '[caption]: tv shows-movies' + '\n'
    for col in df.columns:
        input_data += '|' + col
    input_data += '|\n'
    
    for cell in example:
        input_data += '|' + str(cell)
    input_data += '|\n'
    
    for col in df.columns:
        if col == 'Rating':
            cell_value = '[TO-FILL]'
            ground_truth = row[col].lower().strip()
        else:
            cell_value = row[col]
        input_data += '|' + str(cell_value)
        
    input_data += '|'
    answer_format = "{Rating: ''}"

    input_data = input_data.format(answer_format=answer_format)

    print("---------------------------------------------------")
    print(f"Input: \n{input_data}")

    output = chat(input_data, model="gpt-4", temperature=0.3)

    print(f"Output: \n{output}")
    
    if '```json' in output:
        output = output.replace('```json','')
    if '```' in output:
        output = output.replace('```','')
    # imputed_data = process_output(output)
    imputed_data = ast.literal_eval(output)
    
    imputed_value = imputed_data['Rating']

    if imputed_value.lower().strip() in ground_truth:
        acc += 1
    count += 1
            
    fout = jsonlines.open('./results/tv/GPT4_tv_wo_evidence.jsonl', 'a')
    fout.write({'tuple_id':index, 'input': input_data, 'output':output})
    fout.close()

print(f"Accuracy: {acc/count}")

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

template = '''What's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {answer_format}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''

# template = "You are provided with a table that contains some missing denoted by 'N/A'. Your task is to fill in these missing values by replacing each 'N/A' with the correct value. The output should be a complete table, with the missing values filled in, maintaining the original format, i.e. using '|' to seperate each cell.\n"
tuples = {}

with open('/Users/yichendezaizai/Data_Imputation/data/show_movie/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))

output_data = {}
with jsonlines.open('./results/tv/GPT35_tv_wo_evidence.jsonl', 'r') as f:
    for line in f:
        output_data[int(line['tuple_id'])] = line['output']

# 读取表格数据tv_ret_1.csv
tableData = []
ground_truth = []
df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/show_movie/tv_ret_1.csv')


count, acc = 0, 0

example = ['Avatar: The Last Airbender', '2005', 'TV-Y7', '9.2' , '1']
for index, row in df.iterrows():

    if index not in test_qids or index in processed_tuples:
        continue
    
    ground_truth = row['Rating'].lower().strip()

    output = output_data[index]
    
    # imputed_data = process_output(output)
    imputed_data = ast.literal_eval(output)
    
    imputed_value = imputed_data['Rating']

    if imputed_value.lower().strip() in ground_truth:
        acc += 1
    else:
        print("-----------------")
        print(row)
        print(f"Ground Truth: {ground_truth}")
        print(f"Imputed Value: {imputed_value}")
    count += 1
            

print(f"Accuracy: {acc/count}")

## Data Imputation with retrieved tables

In [ ]:
from collections import defaultdict

topK_results = defaultdict(list)
with open('/Users/yichendezaizai/Data_Imputation/retrieval_results/rerank_results/final_data/show_movie_test.tsv', 'r') as f:
    # with open('/Users/yichendezaizai/Data_Imputation/retrieval_results/first_stage/BM25_top100_res_with_score_show_movie.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, docid, rank, score = line.split('\t')
        if int(rank) > 4:
            continue
        topK_results[int(qid)].append(int(docid))

In [ ]:
collection = {}
with open('/Users/yichendezaizai/Data_Imputation/data/show_movie/collection.tsv', 'r') as f:
    for line in f:
        line = line.strip()
        qid, query = line[:line.find('\t')], line[line.find('\t')+1:]
        collection[int(qid)] = query

In [ ]:
def convert_to_table(serialized_tuple):
    # 分割标题和数据
    caption_split = serialized_tuple.split(' attribute ')
    title = caption_split[0].split(']: ')[1].strip()

    # 提取属性和值
    attributes = caption_split[1:]

    headers = []
    values = []
    sign = 0

    for attribute in attributes:
        attribute_value_split = attribute.split(' value ')
        attribute_name = attribute_value_split[0].strip()
        value = attribute_value_split[1].split(' attribute ')[0].strip()  # 分割可能的下一个属性
        
        headers.append(attribute_name)
        values.append(value)

    # 构建表格
    table = 'caption: ' + title + '\n|' + ' | '.join(headers) + ' |\n|' + ' | '.join(values) + ' |'
    return table


In [ ]:
processed_tuples = []
with open('./results/tv/GPT4_tv_with_retrieved_tuples_by_monoT5.jsonl', 'r') as f: # GPT35_tv_with_retrieved_tuples_by_monoT5
    for line in f:
        line = json.loads(line)
        tuple_id = int(line['tuple_id'])
        processed_tuples.append(tuple_id)

print(len(processed_tuples))

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

template = '''Based on the retrieved tabular data, what's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {'Rating': ''}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''
tuples = {}

with open('/Users/yichendezaizai/Data_Imputation/data/show_movie/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))

# 读取表格数据tv_ret_1.csv
tableData = []
ground_truth = []
df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/show_movie/tv_ret_1.csv')

for index, row in df.iterrows():

    if index not in test_qids or index in processed_tuples:
        continue

    input_data = template + '[caption]: tv shows-movies' + '\n'
    for col in df.columns:
        input_data += '|' + col
    input_data += '|\n'
    
    for col in df.columns:
        if col == 'Rating':
            cell_value = '[TO-FILL]'
            ground_truth = row[col].lower()
        else:
            cell_value = row[col]
        input_data += '|' + str(cell_value)
        
    input_data += '|\n'

    input_data += 'Retrieved Tables:\n'
    retrieved_tables = topK_results[index]
    for rank, docid in enumerate(retrieved_tables):
        input_data += 'Table ' + str(rank+1) + ': ' + convert_to_table(collection[docid]) + '\n'

    print("---------------------------------------------------")
    print(f"Input: \n{input_data}")

    output = chat(input_data, model="gpt-4", temperature=0.3)

    print(f"Output: \n{output}")

    
    fout = jsonlines.open('./results/tv/GPT4_tv_with_retrieved_tuples_by_monoT5.jsonl', 'a')
    fout.write({'tuple_id':index, 'input': input_data, 'output':output})
    fout.close()

### Calculate scores

In [ ]:
import json
import jsonlines
import pandas as pd
import ast

template = '''What's the most likely value for the [TO-FILL] cell in the table below? Please respond using JSON: {answer_format}, the key is attribute name of each [TO-FILL], value is the predicted value for each [TO-FILL].\n'''

# template = "You are provided with a table that contains some missing denoted by 'N/A'. Your task is to fill in these missing values by replacing each 'N/A' with the correct value. The output should be a complete table, with the missing values filled in, maintaining the original format, i.e. using '|' to seperate each cell.\n"
tuples = {}

with open('/Users/yichendezaizai/Data_Imputation/data/show_movie/annotated_data/folds.json', 'r') as f:
    folds = json.load(f)
    test_qids = folds['test']
print(len(test_qids))

output_data = {}
with jsonlines.open('./results/tv/GPT4_tv_with_retrieved_tuples_by_monoT5.jsonl', 'r') as f:
    for line in f:
        output_data[int(line['tuple_id'])] = line['output']

# 读取表格数据tv_ret_1.csv
tableData = []
ground_truth = []
df = pd.read_csv('/Users/yichendezaizai/Data_Imputation/data/show_movie/tv_ret_1.csv')


count, acc = 0, 0

example = ['Avatar: The Last Airbender', '2005', 'TV-Y7', '9.2' , '1']
for index, row in df.iterrows():

    if index not in test_qids or index in processed_tuples:
        continue
    
    ground_truth = row['Rating'].lower().strip()

    output = output_data[index]
    
    # imputed_data = process_output(output)
    imputed_data = ast.literal_eval(output)
    
    imputed_value = imputed_data['Rating']

    if imputed_value.lower().strip() in ground_truth:
        acc += 1
    else:
        print("-----------------")
        print(row)
        print(f"Ground Truth: {ground_truth}")
        print(f"Imputed Value: {imputed_value}")
    count += 1
            

print(f"Accuracy: {acc/count}")